In [ ]:
import re
from typing import Pattern
from dataclasses import dataclass
import pyparsing as pp
pp.ParserElement.enable_Packrat()

In [ ]:
number_regexp =  r"""(?x)                   # a number contains:
                   [0-9]+                   #   one or more digits  
                   (?: \.[0-9]+ )?          #   optionally followed by decimal dot and more digits
                   s?                       #   optionally followed by 's' (e.g. "1990s")
                 """
alpha_regexp =   r"""(?x)                   # an alphanumeric word:
                   (?!                      #   never has index sign prefixes of the form:
                     (?: THUMB- )?          #     THUMB-IX-/THUMB-POSS-/THUMB-SELF-
                     (?:                    #   or of the form
                         IX- |              #     IX-
                         POSS- |            #     POSS-
                         SELF-              #     SELF-
                     )  
                   )                        # and contains:
                   [A-Z0-9]                 #   one uppercase letter or digit
                   (?:                      #   optionally followed by:
                     [A-Z0-9'-]*            #     more uppercase letters and digits, and punctuation characters,
                     [A-Z0-9]               #     where the last character may not be punctuation
                   )?
                   (?:                      # and is optionally followed by:
                     \.                     #   a trailing . (e.g., ETC.)
                   |                        # or
                     :[0-9]                 #   a trailing colon and digit (e.g. COUNT-ON-FINGERS:2)
                   )?
                 """
lookahead_regexp = r"""(?x)                 # a word may not be followed by:
                   (?:
                     (?! [a-z] )            #   any lowercase letters (to prevent e.g. AGE-1p from matching as AGE-1),              
                   |                        # with the exception of:
                     (?= wg )               #   wg, describing a wiggle sign (e.g., MOTHERwg)
                   )
                 """
word_all_regexp = r"""(?x)                  # all put together, a word may be:
                   (?: %s | %s )            #   either a number or an alphanumeric word,
                   %s                       #   not followed by lowercase letters, except for wg
                 """    % (number_regexp, alpha_regexp, lookahead_regexp)

In [ ]:
word = pp.Regex(word_all_regexp)
cl_prefix = pp.one_of(["CL", "DCL", "LCL", "SCL", "BCL", "BPCL", "PCL", "ICL"], as_keyword = True)
ns_prefix = pp.Keyword("ns-")
fs_prefix = pp.Keyword("fs-")
lex_exceptions = pp.one_of(["part", "'WHAT'"], as_keyword = True)
aspect_text = pp.Keyword("aspect")
index_core_ix = pp.Keyword("IX")
other_index_core = pp.one_of(["POSS", "SELF"], as_keyword = True)
person = pp.Regex(r"[0-9]p")
mwe_indic = pp.Keyword("-")
arc = pp.Keyword("-arc")
loc = pp.Keyword("-loc")
compound = pp.Keyword("+")
choice = pp.Keyword("/")
contraction = pp.Keyword("^")

In [ ]:
WORD_BASE = word & (mwe_indic & word)
FS = fs_prefix & WORD_BASE > "FingerSpell"
NS = ns_prefix & WORD_BASE > "NameSign"
LEXICAL = lex_exceptions | WORD_BASE
SIGN_CORE = NS | FS | LEXICAL
CONTRACTION_UNIT = contraction + SIGN_CORE
SIGN_UNIT = SIGN_CORE + pp.Optional(CONTRACTION_UNIT)
COMPOUND = compound + SIGN_UNIT
CHOICE = SIGN_UNIT + choice + SIGN_UNIT
CLASSIFIER = cl_prefix
INDEX = (index_core_ix | other_index_core) + "-" + person
INDEX_LOC_NUC = INDEX + loc
INDEX_LOC = arc

ALLSIGNS = (CLASSIFIER | INDEX_LOC | INDEX | COMPOUND | CHOICE | SIGN_UNIT | CONTRACTION_UNIT) + pp.StringEnd()


In [ ]:
@dataclass
class GlossExpressions:
    """A collection of valid expressions used in gloss conventions."""
    
    number: Pattern[str] = re.compile(number_regexp, re.X)
    alpha: Pattern[str] = re.compile(alpha_regexp, re.X)
    lookahead: Pattern[str] = re.compile(lookahead_regexp, re.X)
    word_all: Pattern[str] = re.compile(word_all_regexp, re.X)


In [ ]:
@dataclass
class RegexRule:
    # Guarantees that any combination is a valid Regex gloss morpheme.
    name: str
    pattern: str
    flags: int = re.X

    def __post_init__(self):
        self._re = re.compile(self.pattern, self.flags)
        
    @classmethod

    def __or__(self, rule: "RegexRule") -> "RegexRule":
        """Allow only number|alpha, not arbitrary unions."""
        allowed = {("number","alpha"), ("alpha","number")}
        if (self.name, rule.name) not in allowed:
            raise ValueError(f"Cannot `or` {self.name} with {rule.name}")
        pat = f"(?:{self.pattern}|{rule.pattern})"
        return RegexRule(f"{self.name}|{rule.name}", pat, self.flags)

    def then(self, rule: "RegexRule") -> "RegexRule":
        """Regular concatenation—but only allow number/alpha → lookahead."""
        if rule.name != "lookahead":
            raise ValueError(f"{self.name} can only be followed by lookahead, not {rule.name}")
        pat = f"{self.pattern}{rule.pattern}"
        return RegexRule(f"{self.name}+{rule.name}", pat, self.flags)

    def compile(self) -> Pattern:
        return self._re